In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import sknetwork as skn
import networkx as nx
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")

# Globais

In [4]:
NETWORK_HRPD_BASE_PATH = Path("/Users/alfredoguilhermedasilvasouza/IdeaProjects/acdbio-mlnet-driver-discovery/acdbio_mlnet_driver_discovery/data/networks/silver")
MAF_BASE_PATH = Path("/Users/alfredoguilhermedasilvasouza/IdeaProjects/acdbio-mlnet-driver-discovery/acdbio_mlnet_driver_discovery/data/mafs/gold")
INTOGEN_BASE_PATH = Path("/Users/alfredoguilhermedasilvasouza/IdeaProjects/acdbio-mlnet-driver-discovery/acdbio_mlnet_driver_discovery/data/benchmarks/intogen")
NETWORK_HRPD_FEATURES_BASE_PATH = Path("/Users/alfredoguilhermedasilvasouza/IdeaProjects/acdbio-mlnet-driver-discovery/acdbio_mlnet_driver_discovery/data/network_features")
CGC_BASE_PATH = Path("/Users/alfredoguilhermedasilvasouza/IdeaProjects/acdbio-mlnet-driver-discovery/acdbio_mlnet_driver_discovery/data/benchmarks/CGC")
FALSE_POSITIVE_BASE_PATH = Path("/Users/alfredoguilhermedasilvasouza/IdeaProjects/acdbio-mlnet-driver-discovery/acdbio_mlnet_driver_discovery/data/benchmarks/false_positives")

In [5]:
NETWORK_HPRD_PATH = NETWORK_HRPD_BASE_PATH.joinpath("HPRD.txt")
MAF_BRCA_PATH = MAF_BASE_PATH.joinpath("brca_data_mutations_extended.parquet")
INTOGEN_BRCA_PATH = INTOGEN_BASE_PATH.joinpath("IntOGen-DriverGenes_BRCA_TCGA.tsv")
NETWORK_FEATURES_HPRD_PATH = NETWORK_HRPD_FEATURES_BASE_PATH.joinpath("HPRD_features.parquet")
CGC_PATH = CGC_BASE_PATH.joinpath("CGC_COSMIC_Census_allTue Jul 27 01_37_31 2021.tsv")
FALSE_POSITIVE_PATH = FALSE_POSITIVE_BASE_PATH.joinpath("false_positives_NCG6.txt")

### Lista de arestas

In [6]:
edges = pd.read_csv(NETWORK_HPRD_PATH, sep=' ', header=None, names=['Gene1', 'Gene2'])

In [7]:
edges

,Gene1,Gene2
0,CHRNA1,CHRNE
1,CHRNA1,CHRNG
2,CHRNA1,CHRND
3,FHL2,AR
4,FHL2,HAND1
...,...,...
24345,RRAGC,RRAGA
24346,TSPO,DBI
24347,TSPO,BZRAP1
24348,ERVW-1,SLC1A4


### 1. Lista de Genes e Mapeamento Gene-Índice
Definir a lista de genes:

In [8]:
# Lista de genes
genes = ['BRCA1', 'TP53', 'EGFR', 'MYC', 'PTEN']

In [9]:
# Mapeamento do nome do gene para o índice do nó
gene_to_index = {gene: idx for idx, gene in enumerate(genes)}

### 2. Definir as Arestas com Base nos Genes
Exemplo de arestas entre genes:

Suponha que temos as seguintes interações entre genes:

BRCA1 interage com TP53
TP53 interage com EGFR
EGFR interage com MYC
MYC interage com PTEN
Definir as arestas usando os índices correspondentes:


In [10]:
# Lista de arestas entre genes (pares de genes)
edges = [
    ('BRCA1', 'TP53'),
    ('TP53', 'EGFR'),
    ('EGFR', 'MYC'),
    ('MYC', 'PTEN')
]

# Converter as arestas para índices
edge_index = torch.tensor([
    [gene_to_index[edge[0]] for edge in edges],  # de
    [gene_to_index[edge[1]] for edge in edges]   # para
], dtype=torch.long)


In [11]:
edges

[('BRCA1', 'TP53'), ('TP53', 'EGFR'), ('EGFR', 'MYC'), ('MYC', 'PTEN')]

In [12]:
edge_index

tensor([[0, 1, 2, 3],
        [1, 2, 3, 4]])

### 3. Associar Atributos aos Genes
Definir os atributos para cada gene:

Vamos criar dicionários que mapeiam cada gene ao seu atributo correspondente.

In [13]:
# Atributos dos genes
is_driver_dict = {
    'BRCA1': 1,
    'TP53': 1,
    'EGFR': 0,
    'MYC': 0,
    'PTEN': 1
}

is_false_positive_dict = {
    'BRCA1': 0,
    'TP53': 0,
    'EGFR': 1,
    'MYC': 0,
    'PTEN': 0
}

found_in_intogen_dict = {
    'BRCA1': 1,
    'TP53': 1,
    'EGFR': 0,
    'MYC': 1,
    'PTEN': 1
}

### Converter os atributos em tensores alinhados aos índices dos genes:

In [15]:
# Número de genes
num_genes = len(genes)

# Inicializar tensores vazios para os atributos
is_driver = torch.zeros((num_genes, 1), dtype=torch.float)
is_false_positive = torch.zeros((num_genes, 1), dtype=torch.float)
found_in_intogen = torch.zeros((num_genes, 1), dtype=torch.float)

# Preencher os tensores com os valores correspondentes
for gene, idx in gene_to_index.items():
    is_driver[idx] = is_driver_dict[gene]
    is_false_positive[idx] = is_false_positive_dict[gene]
    found_in_intogen[idx] = found_in_intogen_dict[gene]

# Concatenar os atributos para formar a matriz de features dos nós
x = torch.cat([is_driver, is_false_positive, found_in_intogen], dim=1)


### 4. Criar o Objeto de Dados do Grafo
Definir os rótulos (y):

Vamos supor que queremos prever se um gene é um driver (`is_driver`).

In [16]:
# Labels para treinamento (se é driver)
y = is_driver.squeeze().long()  # Converter para inteiro longo


Criar o objeto `Data`:

In [18]:
from torch_geometric.data import Data

data = Data(x=x, edge_index=edge_index, y=y)


#### Adicionar o mapeamento de índices para genes (opcional para referência):

Embora o PyTorch Geometric não use diretamente os nomes dos genes, podemos manter o mapeamento para referência futura.

In [19]:
# Inverter o mapeamento para obter index_to_gene
index_to_gene = {idx: gene for gene, idx in gene_to_index.items()}


### 5. Definir o Modelo GNN
O modelo permanece semelhante ao anterior.

In [22]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNClassifier, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)


Instanciar o modelo

In [23]:
input_dim = x.shape[1]    # Número de atributos de entrada (3 neste caso)
hidden_dim = 16           # Tamanho da camada oculta
output_dim = 2            # Número de classes (driver ou não driver)

model = GCNClassifier(input_dim, hidden_dim, output_dim)


### 6. Treinar e Avaliar o Modelo
Definir o otimizador e a função de perda:

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()


#### Treinar o modelo:

In [27]:
model.train()
for epoch in range(100):  # Número de épocas de treinamento
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Época {epoch}, Loss: {loss.item():.4f}')


Época 0, Loss: 0.6676
Época 10, Loss: 0.5837
Época 20, Loss: 0.5377
Época 30, Loss: 0.4825
Época 40, Loss: 0.4276
Época 50, Loss: 0.3697
Época 60, Loss: 0.3151
Época 70, Loss: 0.2629
Época 80, Loss: 0.2159
Época 90, Loss: 0.1745


#### Avaliar o modelo:

In [28]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int((pred == data.y).sum())
accuracy = correct / num_genes

print(f'Acurácia: {accuracy * 100:.2f}%')


Acurácia: 100.00%


#### Exibir as predições com os nomes dos genes:

In [30]:
for idx, gene in index_to_gene.items():
    print(f'Gene: {gene}, Verdadeiro: {data.y[idx].item()}, Predito: {pred[idx].item()}')

Gene: BRCA1, Verdadeiro: 1, Predito: 1
Gene: TP53, Verdadeiro: 1, Predito: 1
Gene: EGFR, Verdadeiro: 0, Predito: 0
Gene: MYC, Verdadeiro: 0, Predito: 0
Gene: PTEN, Verdadeiro: 1, Predito: 1
